# MoSDeF: A Molecular Simulation and Design Framework

The Molecular Simulation and Design Framework (MoSDeF) is a collection of open-source tools ([hosted on Github](https://github.com/mosdef-hub)) aimed at facilitating the construction and simulation of complex molecular systems - with a particular focus on the automated screening of large structural parameter spaces. All tools are written as Python packages and feature a Python-based API.

## Basic Foyer tutorial

The second of the MoSDeF tools we will explore is the [Foyer package](https://github.com/mosdef-hub/foyer) which provides a means for the automated application and dissemination of forcefields. This tool can be used to take a molecular model featuring particle positions and bonds (such as an mBuild `Compound`) and automatically perform atomtyping (as well as parameterization) as directed by a user-specified forcefield. In this tutorial, we'll demonstrate how Foyer works by using it to perform atomtyping and parameterization of several alkane molecules.

### Foyer XML format

Foyer forcefields are defined within an XML file that contains both the 'rules' required for atomtyping as well as the forcefield parameters within a single file. The Foyer XML format is an extension of the [OpenMM forcefield XML format](http://docs.openmm.org/7.0.0/userguide/application.html#creating-force-fields), which you may already be familiar with. The only differences reside in the `AtomTypes` section, where several additional attributes are available (which we will examine in a moment) for each `Type` to allow for atomtyping.

Let's take a look at the basic structure of a Foyer XML file.

#### AtomTypes

In [ ]:
! sed -n 34,40p OPLSaa_perfluoroalkanes.xml

The `AtomTypes` section of the Foyer XML is similar to that used for OpenMM forcefield XMLs; however, each `Type` in Foyer XML supports four additional attributes not found in OpenMM:
* `def` - SMARTS string describing the chemical substructure of this atomtype (Follow [this link](https://github.com/mosdef-hub/foyer/blob/master/docs/smarts.md) for more on SMARTS-based atomtyping using Foyer.)
* `desc` - Brief description of the atomtype
* `doi` - DOI reference for parameters associated with this atomtype
* `overrides` - One or more atomtypes to 'override', providing precedence to this atomtype

Atomtypes are defined using [SMARTS](http://www.daylight.com/dayhtml/doc/theory/theory.smarts.html), which provide a language for describing chemical structures and substructures.

#### Forcefield parameters

The remaining sections of the Foyer XML are currently the same as defined in the OpenMM format and feature both bonded and nonbonded forcefield parameters.

In [ ]:
! sed -n 41,62p OPLSaa_perfluoroalkanes.xml

### Constructing a Foyer XML

Now we'll work to construct a valid Foyer forcefield XML that can correctly parameterize linear alkanes according to the OPLS forcefield. We've provided a template XML forcefield file, where only SMARTS strings are missing and need to be defined.

Included in this example are MOL2 files for three alkane molecules (methane, ethane, and propane) that contain information as to the correct atomtypes that should be assigned to each atom, which we will use for testing and validating the forcefield file.

We'll quickly visualize the three molecules we will be atomtyping.

In [ ]:
%matplotlib notebook
import mbuild as mb
from visualize import visualize
methane = mb.load('test_molecules/methane.mol2')
ethane = mb.load('test_molecules/ethane.mol2')
propane = mb.load('test_molecules/propane.mol2')
ethane.translate_to([0.6, 0, 0])
propane.translate_to([1, 0, 0])
alkanes = mb.Compound(subcompounds=(methane, ethane, propane))
visualize(alkanes)

In the `'forcefield.xml'` template, the `def` attribute of each carbon atomtype features an arbitrary string rather than the correct SMARTS. We will have to correctly define the SMARTS for these atomtypes for Foyer to be able to successfully parameterize each of these alkanes.

A script is provided, `test_atomtyping.py`, which will use our Foyer XML to attempt to atomtype each of the three alkane molecules and will compare the resulting atomtypes assigned by Foyer to the known atomtypes that should be assigned (read from the MOL2 files). We can use the [pytest utility](https://docs.pytest.org/en/latest/) to perform these tests.

Let's go ahead and use pytest to run `test_atomtyping.py` and see what happens.

In [ ]:
! pytest -v test_atomtyping.py

Unsurprisingly, all three of our tests failed as Foyer was unable to identify any valid atomtypes for the atoms in each of our molecules. This is because none of the currently defined SMARTS for the carbon atomtypes are correct. Let's see what happens if we change the SMARTS strings for the carbons.

In SMARTS you can define a carbon with four neighbors as `'[C;X4]'`.

To edit your forcefield.xml file go back to the "Home" window that was launched when the "jupyter notebook" command was first executed and follow the steps below.
<img src="editing-forcefield.png" alt="Drawing" style="width: 850px"/>

In [ ]:
! pytest -v test_atomtyping.py

Again all three tests failed, this time because we can't differentiate between the three OPLS carbons.

Now let's distinguish these by adding to the SMARTS the number of hydrogen neighbors each OPLS carbon should have.

In [ ]:
! pytest -v test_atomtyping.py

We still have multiple types found for carbons in each molecule. However, we can add precendence to our atomtypes through the `overrides` attribute.

In [ ]:
! pytest -v test_atomtyping.py

Alternatively, we can add specificity to the SMARTS without overrides, by explicitly defining all four neighbors for each atomtype. However, in some cases `overrides` are necessary to distinguish atomtypes that have the same local environment (e.g. a methyl group in toluene vs. in an alkane).
<img src="ch3-toluene.png" alt="Drawing" style="width: 850px"/>

In [ ]:
! pytest -v test_atomtyping.py

### Foyer + mBuild

As we saw in the mBuild tutorial, Foyer has been designed to integrate seamlessly with mBuild, although it can also be used as a standalone package. To call Foyer from mBuild all a user needs to do is specify the forcefield XML file to use by passing either the `forcefield_name` or `forcefield_files` flag to `Compound.save()`. Internally this will perform parameterization of the mBuild `Compound` using Foyer.

Additionally, the `references_file` flag can be used to output a file (in BibTeX format) containing the references for the parameters used for parameterization of this `Compound`.

Here, we explore this functionality by loading a simple methane molecule into an mBuild `Compound` and save using the forcefield we've just defined.

In [ ]:
import mbuild as mb

ch4 = mb.load('ch4.pdb')
ch4.save('ch4.top', forcefield_files='forcefield_master.xml', references_file='ch4.bib', overwrite=True)

In [ ]:
help(mb.Compound.save)

In [ ]:
! cat ch4.bib

## Quick-start

If you are interested in defining your existing forcefield(s) in the Foyer XML format, we currently have a forcefield template hosted on Github to help you get started - https://github.com/mosdef-hub/forcefield_template, that includes some basic scripts for validation and error checking. We also refer interested users to the [OpenMM documentation](http://docs.openmm.org/7.0.0/userguide/application.html#creating-force-fields) for detailed instructions on forcefield creation.